In [1]:
## notes
#  create readme
#  some fy18 finance columns aren't as updated as the ipeds manual search
#  group multi-campuses

import pandas as pd
import glob
import numpy as np
import os
from pandas import ExcelWriter
from IPython.display import display

pd.options.mode.chained_assignment = None

path_main = os.getcwd() + "\Main CSV\*.csv"
path_distance = os.getcwd() + "\Additional CSV\Distance\*.csv"
path_employee = os.getcwd() + "\Additional CSV\Employee\*.csv"
path_private = os.getcwd() + "\Private Univ\*.csv"

df_appended=pd.DataFrame()
df2=pd.DataFrame()
df3=pd.DataFrame()
    
    
# cleaning main csv's

for fname in glob.glob(path_main):
    df = pd.read_csv(fname)

    df['Key'] = df['unitid'].map(str) + ' ' + df['year'].map(str)
    
    df.columns = df.columns.str.replace(r'\d+', '')
    
    df.columns = df.columns.str.replace(r'_RV', '')
    df.columns = df.columns.str.replace(r'F_FA.Total expenses  deductions - Current year total', 'F_FA.Total expenses and deductions - Current year total')
    
    df['Total Expenses'] = df[["F_FA.Total expenses and deductions - Current year total", "F_F.Total expenses-Total amount"]].max(axis=1)
    df['Construction Expenses'] = df[["F_FA.Construction in progress - Ending balance", "F_F.Construction in Progress"]].max(axis=1)
    df['Interest Expenses'] = df[["F_FA.Total expenses deductions - Interest", "F_F.Total expenses-Interest"]].max(axis=1)
    df['Other Expenses'] = df[["F_FA.Other expenses  deductions - Current year total", "F_F.Other expenses-Total amount"]].max(axis=1)
    
    df.drop(columns=['F_FA.Construction in progress - Ending balance', 'F_FA.Other expenses  deductions - Current year total',
                     'F_FA.Total expenses and deductions - Current year total', 'F_FA.Total expenses deductions - Interest',
                     'F_F.Construction in Progress', 'F_F.Other expenses-Total amount',
                     'F_F.Total expenses-Total amount','F_F.Total expenses-Interest'],inplace=True)

    df.rename(columns={'DRVEF.Total  enrollment': 'Fall Student Headcount', 
                       'DRVEF.Full-time undergraduate enrollment': 'Full-time Undergraduate Enrollment',
                       'DRVEF.Full-time graduate enrollment': 'Full-time Graduate Enrollment', 
                       'DRVEF.Part-time undergraduate enrollment': 'Part-time Undergraduate Enrollment',
                       'DRVEF.Part-time graduate enrollment': 'Part-time Graduate Enrollment', 
                       'DRVHR.Total FTE staff': 'Total FTE Staff',
                       'DRVEF.-month unduplicated headcount, total: -': '12 Month Unduplicated Headcount',
                       'DRVEF.-month full-time equivalent enrollment: -': '12 Month FTE Enrollment',
                       'unitid': 'UnitID',
                       'institution name': 'Institution Name',
                       'year': 'Year'},
                       inplace=True)
    
    df_appended = df_appended.append(df, sort = True)
    
    
# cleaning employment csv's

for fname in glob.glob(path_employee):
    df = pd.read_csv(fname)
    
    df['Key'] = df['unitid'].map(str) + ' ' + df['year'].map(str)
    
    df.columns = df.columns.str.replace(r'\d+', '')
    df.columns = df.columns.str.replace(r'_RV', '')
    df.rename(columns={'EAP.Full-time employees':'Full-time Employees',
                       'EAP.Part-time employees':'Part-time Employees'}, inplace=True)

    indexNames = df[df['EAP.Occupation and faculty/tenure status']!= 'All staff'].index
    df.drop(indexNames , inplace=True)
    df = df[['Key','Full-time Employees','Part-time Employees']]

    df = df_appended.merge(df, on='Key')
    df2 = df2.append(df)

df_appended = df2


# cleaning distance csv's

for fname in glob.glob(path_distance):
    df = pd.read_csv(fname)
    
    df['Key'] = df['unitid'].map(str) + ' ' + df['year'].map(str)
    
    df.columns = df.columns.str.replace(r'\d+', '')
    df.columns = df.columns.str.replace(r'_RV', '')
    
    indexNames = df[df['EFA_DIST.Level of student']!= 'All students total'].index
    df.drop(indexNames , inplace=True)
    df = df[['Key','EFA_DIST.Students enrolled exclusively in distance education courses']]
    df.rename(columns={'EFA_DIST.Students enrolled exclusively in distance education courses': "Distance Course Enrollment"}, inplace=True)

    df = df_appended.merge(df, on='Key')
    df3 = df3.append(df)

df_appended = df3


# reorganizing columns with different fy/year combo
df4 = pd.DataFrame()
df4 = df_appended[['Key', 'UnitID', 'Institution Name', 'Year', 'Fall Student Headcount','Full-time Undergraduate Enrollment', 
                   'Full-time Graduate Enrollment', 'Part-time Undergraduate Enrollment', 'Part-time Graduate Enrollment',
                   'Total FTE Staff', 'Full-time Employees', 'Part-time Employees', 'Distance Course Enrollment']]

df4['Year'] = df4['Year'] + 1 
df4['Key'] = df4['UnitID'].map(str) + ' ' + df4['Year'].map(str)

df5 = df_appended[['12 Month Unduplicated Headcount',
                  '12 Month FTE Enrollment', 'Key', 'Total Expenses',
                  'Construction Expenses', 'Interest Expenses', 'Other Expenses']]

df6 = pd.merge(df4,df5, on='Key', how = 'left')
df6 = df6.sort_values(['Year'], ascending = True)


# labeling private universities
priv_csv = glob.glob(path_private)
df7 = pd.read_csv(priv_csv[0])
df7 = df7[['unitid']]
df7['Public'] = 'Private'
df7.rename(columns={'unitid': "UnitID"}, inplace = True)
df8 = df6.merge(df7, on = 'UnitID', how = 'left')
df8['Public'].fillna(value = 'Public', inplace = True)


# calculated columns
df8['Operating Expenses'] = df8['Total Expenses'] - df8['Interest Expenses'] - df8['Other Expenses']

df8['Fall Student FTE'] = np.where(df8['Public'] == 'Public', 
                                   df8['Full-time Undergraduate Enrollment'] + (df8['Part-time Undergraduate Enrollment'] * 0.403543) + df8['Full-time Graduate Enrollment'] + (df8['Part-time Graduate Enrollment'] * 0.361702),
                                   df8['Full-time Undergraduate Enrollment'] + (df8['Part-time Undergraduate Enrollment'] * 0.392857) + df8['Full-time Graduate Enrollment'] + (df8['Part-time Graduate Enrollment'] * 0.382059))
df8['Fall Student FTE'] = df8['Fall Student FTE'].round(2)

df8['Total Employees'] = df8['Full-time Employees'] + df8['Part-time Employees']


# renaming and dropping columns
df8 = df8[['Institution Name','Year','Public','Fall Student Headcount','12 Month Unduplicated Headcount','12 Month FTE Enrollment',
           'Fall Student FTE','Full-time Undergraduate Enrollment', 'Full-time Graduate Enrollment',
           'Part-time Undergraduate Enrollment', 'Part-time Graduate Enrollment','Distance Course Enrollment',
           'Total FTE Staff', 'Total Employees', 'Full-time Employees', 'Part-time Employees','Construction Expenses','Operating Expenses', 'Interest Expenses', 'Total Expenses','Other Expenses']]


# writing to excel
writer = ExcelWriter('Analysis Factors1.xlsx')
df8.to_excel(writer,'Sheet1', index = False)
writer.save()